# Data Science Championship - South Zone

> "A Multi Class Classification problem, for classifying the diseases in the Paddy Plant"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [paddy, plant, disease, classification, multi, class, kaggle, convolutional, neural, network]
- hide: false

In [1]:
# Installing the modules

!pip3 install optuna
!pip3 install category_encoders

     |████████████████████████████████| 308 kB 4.2 MB/s 
     |████████████████████████████████| 210 kB 17.6 MB/s 
     |████████████████████████████████| 81 kB 7.7 MB/s 
     |████████████████████████████████| 78 kB 5.8 MB/s 
     |████████████████████████████████| 112 kB 54.0 MB/s 
     |████████████████████████████████| 146 kB 37.4 MB/s 
     |████████████████████████████████| 49 kB 4.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=66c2db253cdea3f9f1e8e393374f1d74da8ccd92d7174a8fae7c6d6e8c8b5844
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
     |████████████████████████████████| 80 kB 3.5 MB/s 


In [2]:
# Required modules

import shutil
import optuna
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import category_encoders as ce

from xgboost import XGBRegressor

from google.colab import drive
from matplotlib import pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# Config

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 12)
pd.set_option('display.max_columns', None)

In [4]:
# Mounting the drive

drive.mount('./mydrive')

Mounted at ./mydrive


In [5]:
# Moving files to workspace

shutil.copy('/content/mydrive/MyDrive/Machine Hack/Data Science Students Championship/train.csv', './train.csv')
shutil.copy('/content/mydrive/MyDrive/Machine Hack/Data Science Students Championship/test.csv', './test.csv')
shutil.copy('/content/mydrive/MyDrive/Machine Hack/Data Science Students Championship/submission.csv', './submission.csv')

'./submission.csv'

In [6]:
# Load the data

train = pd.read_csv('train.csv')
train.head()

,Property_ID,room,layout_type,property_type,locality,price,area,furnish_type,bathroom,city,parking_spaces,floor,pet_friendly,power_backup,washing_machine,air_conditioner,geyser/solar,security_deposit,CCTV/security,lift,neighbourhood
0,42208,3,BHK,Independent House,Palavakkam,33624,1312,Furnished,2,Chennai,1,1,1,0,0,1,0,302616,0,0,300
1,90879,1,BHK,Apartment,Manikonda,9655,1474,Unfurnished,2,Hyderabad,0,17,0,1,0,0,1,19310,0,1,1600
2,99943,3,BHK,Apartment,Jodhpur Park,23699,1837,Semi-Furnished,2,Kolkata,0,10,1,1,1,1,0,118495,0,1,3100
3,113926,1,BHK,Apartment,Chembur,6306,606,Unfurnished,1,Mumbai,0,18,0,0,0,0,0,37836,0,1,300
4,185438,1,BHK,Studio Apartment,Kalewadi Pandhapur Road,12008,498,Semi-Furnished,3,Pune,0,14,0,0,1,1,0,72048,0,1,0


In [7]:
# Inspect the data

train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134683 entries, 0 to 134682
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Property_ID       134683 non-null  int64 
 1   room              134683 non-null  int64 
 2   layout_type       134683 non-null  object
 3   property_type     134683 non-null  object
 4   locality          134683 non-null  object
 5   price             134683 non-null  int64 
 6   area              134683 non-null  int64 
 7   furnish_type      134683 non-null  object
 8   bathroom          134683 non-null  int64 
 9   city              134683 non-null  object
 10  parking_spaces    134683 non-null  int64 
 11  floor             134683 non-null  int64 
 12  pet_friendly      134683 non-null  int64 
 13  power_backup      134683 non-null  int64 
 14  washing_machine   134683 non-null  int64 
 15  air_conditioner   134683 non-null  int64 
 16  geyser/solar      134683 non-null  int

,Property_ID,room,price,area,bathroom,parking_spaces,floor,pet_friendly,power_backup,washing_machine,air_conditioner,geyser/solar,security_deposit,CCTV/security,lift,neighbourhood
count,134683.000000,134683.000000,134683.000000,134683.000000,134683.000000,134683.000000,134683.000000,134683.000000,134683.000000,134683.000000,134683.000000,134683.000000,1.346830e+05,134683.000000,134683.000000,134683.000000
mean,96036.100777,2.029677,36690.033894,1480.388490,2.040488,0.534388,9.163087,0.527602,0.337051,0.472561,0.692626,0.440137,2.202480e+05,0.561838,0.595851,2033.024212
std,55565.228125,0.937308,62620.364025,1412.464718,0.867065,0.498818,5.957549,0.499239,0.472704,0.499248,0.461407,0.496405,4.204503e+05,0.496163,0.490728,1159.635981
min,2.000000,1.000000,1583.000000,81.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.180000e+03,0.000000,0.000000,0.000000
25%,47940.000000,1.000000,12035.500000,759.000000,1.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.580250e+04,0.000000,0.000000,1100.000000
50%,95950.000000,2.000000,20856.000000,1114.000000,2.000000,1.000000,9.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.142640e+05,1.000000,1.000000,2000.000000
75%,144194.500000,3.000000,36014.000000,1580.000000,2.000000,1.000000,15.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.207045e+05,1.000000,1.000000,3000.000000
max,192405.000000,5.000000,799325.000000,13942.000000,5.000000,1.000000,19.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.940780e+06,1.000000,1.000000,4000.000000


In [8]:
# Load the test data

test = pd.read_csv('./test.csv')
test.head()

,Property_ID,room,layout_type,property_type,locality,area,furnish_type,bathroom,city,parking_spaces,floor,pet_friendly,power_backup,washing_machine,air_conditioner,geyser/solar,security_deposit,CCTV/security,lift,neighbourhood,price
0,114342,2,BHK,Independent Floor,Palava,1347,Semi-Furnished,1,Mumbai,0,2,0,1,1,1,0,72624,1,0,900,NaN
1,88819,1,BHK,Independent House,Somajiguda,634,Semi-Furnished,3,Hyderabad,1,4,0,0,1,1,0,19656,0,0,2500,NaN
2,85623,1,BHK,Apartment,Toli Chowki,524,Unfurnished,1,Hyderabad,1,3,1,1,0,0,0,7500,0,0,3200,NaN
3,130856,3,BHK,Apartment,Thane West,1837,Unfurnished,5,Mumbai,1,9,1,0,0,0,1,137646,1,1,1200,NaN
4,40089,2,BHK,Apartment,Krishnarajapura,1208,Semi-Furnished,2,Bangalore,1,17,0,1,1,1,0,110898,0,1,1000,NaN


In [9]:
# Checking for the missing values

if train.isna().any().any():
    print(train.isna().any())
else:
    print("No Missing Values")

No Missing Values


## Feature Engineering

In [10]:
# Converting the categorical variables

train['layout_type'] = np.where(train['layout_type'] == 'BHK', 1, 0)
test['layout_type'] = np.where(test['layout_type'] == 'BHK', 1, 0)

for col in train.columns:
    if train[col].dtype == 'object':
        encoder = ce.cat_boost.CatBoostEncoder()

        train[col] = encoder.fit_transform(train[col], train['price'])
        test[col] = encoder.transform(test[col])

In [11]:
# Seperating out features and labels

X = train.drop(['Property_ID', 'price'], axis=1)
y = train['price']
X_test = test.drop(['Property_ID', 'price'], axis=1)

In [12]:
# Scaling the train and test values

scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)
X_test_sclaed = scaler.transform(X_test)

In [13]:
# Train and test split

X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=88)

## Model Building

In [23]:
# Objective function of lgbm

def objective_lgbm(trial):
    params = dict()
    params['n_estimators'] = trial.suggest_int('n_estimators', 100, 5000, step=13)
    params['learning_rate'] = trial.suggest_float('learning_rate', 0.1, 0.4, step = 0.1)
    params['max_depth'] = trial.suggest_int('max_depth', 1, 5, step=1)
    params['subsample']= trial.suggest_float('subsample', 0.3, 0.8, step=0.1 )
    params['colsample_bytree'] = trial.suggest_float('colsample_bytree', 0.3, 0.8, step = 0.1)
    params['objective'] = 'regression'

    model = lgb.LGBMRegressor(**params, n_jobs=-1)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[lgb.early_stopping(30)], eval_metric="rmse")

    return mean_squared_error(model.predict(X_valid), y_valid, squared=False)

In [24]:
# Fitting the optuna

study = optuna.create_study(direction='minimize')
study.optimize(objective_lgbm, n_trials=100)

[I 2022-05-15 04:21:52,885] A new study created in memory with name: no-name-5078bb55-96d2-4272-bdce-7b7dc902b70f
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:563: UserWarning: The distribution is specified by [100, 5000] and step=13, but the range is not divisible by `step`. It will be replaced by [100, 4988].
  low=low, old_high=old_high, high=high, step=step


[1]	valid_0's rmse: 61495.8	valid_0's l2: 3.78173e+09
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's rmse: 61474.1	valid_0's l2: 3.77906e+09
[3]	valid_0's rmse: 61436.6	valid_0's l2: 3.77446e+09
[4]	valid_0's rmse: 61412.2	valid_0's l2: 3.77146e+09
[5]	valid_0's rmse: 55098.7	valid_0's l2: 3.03587e+09
[6]	valid_0's rmse: 50239.2	valid_0's l2: 2.52398e+09
[7]	valid_0's rmse: 46453.9	valid_0's l2: 2.15797e+09
[8]	valid_0's rmse: 43360	valid_0's l2: 1.88009e+09
[9]	valid_0's rmse: 43347.6	valid_0's l2: 1.87901e+09
[10]	valid_0's rmse: 43339.2	valid_0's l2: 1.87829e+09
[11]	valid_0's rmse: 40901.3	valid_0's l2: 1.67291e+09
[12]	valid_0's rmse: 40880.4	valid_0's l2: 1.67121e+09
[13]	valid_0's rmse: 40867.4	valid_0's l2: 1.67015e+09
[14]	valid_0's rmse: 38752.8	valid_0's l2: 1.50178e+09
[15]	valid_0's rmse: 37072.3	valid_0's l2: 1.37436e+09
[16]	valid_0's rmse: 37062.1	valid_0's l2: 1.3736e+09
[17]	valid_0's rmse: 37055.3	valid_0's l2: 1.3731e+09
[18]	valid_0's r

[I 2022-05-15 04:21:55,040] Trial 0 finished with value: 28192.651315971336 and parameters: {'n_estimators': 2349, 'learning_rate': 0.2, 'max_depth': 1, 'subsample': 0.7, 'colsample_bytree': 0.4}. Best is trial 0 with value: 28192.651315971336.
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:563: UserWarning: The distribution is specified by [100, 5000] and step=13, but the range is not divisible by `step`. It will be replaced by [100, 4988].
  low=low, old_high=old_high, high=high, step=step


[1]	valid_0's rmse: 61446.6	valid_0's l2: 3.77568e+09
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's rmse: 61405.6	valid_0's l2: 3.77064e+09
[3]	valid_0's rmse: 61369	valid_0's l2: 3.76616e+09
[4]	valid_0's rmse: 61347.6	valid_0's l2: 3.76353e+09
[5]	valid_0's rmse: 48860.8	valid_0's l2: 2.38737e+09
[6]	valid_0's rmse: 40653.9	valid_0's l2: 1.65274e+09
[7]	valid_0's rmse: 35526.5	valid_0's l2: 1.26213e+09
[8]	valid_0's rmse: 32379.6	valid_0's l2: 1.04844e+09
[9]	valid_0's rmse: 32377.2	valid_0's l2: 1.04828e+09
[10]	valid_0's rmse: 32374.5	valid_0's l2: 1.04811e+09
[11]	valid_0's rmse: 30636.5	valid_0's l2: 9.38592e+08
[12]	valid_0's rmse: 30634	valid_0's l2: 9.3844e+08
[13]	valid_0's rmse: 30632.1	valid_0's l2: 9.38328e+08
[14]	valid_0's rmse: 29693.6	valid_0's l2: 8.81711e+08
[15]	valid_0's rmse: 29043.5	valid_0's l2: 8.43526e+08
[16]	valid_0's rmse: 29039.6	valid_0's l2: 8.43298e+08
[17]	valid_0's rmse: 29039.1	valid_0's l2: 8.4327e+08
[18]	valid_0's rms

[I 2022-05-15 04:21:56,336] Trial 1 finished with value: 28138.21792163869 and parameters: {'n_estimators': 958, 'learning_rate': 0.30000000000000004, 'max_depth': 2, 'subsample': 0.8, 'colsample_bytree': 0.4}. Best is trial 1 with value: 28138.21792163869.
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:563: UserWarning: The distribution is specified by [100, 5000] and step=13, but the range is not divisible by `step`. It will be replaced by [100, 4988].
  low=low, old_high=old_high, high=high, step=step


...

[1]	valid_0's rmse: 61495.8	valid_0's l2: 3.78173e+09
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's rmse: 61474.1	valid_0's l2: 3.77906e+09
[3]	valid_0's rmse: 55166.7	valid_0's l2: 3.04337e+09
[4]	valid_0's rmse: 55125.6	valid_0's l2: 3.03883e+09
[5]	valid_0's rmse: 50266.1	valid_0's l2: 2.52668e+09
[6]	valid_0's rmse: 46483.7	valid_0's l2: 2.16074e+09
[7]	valid_0's rmse: 43388.2	valid_0's l2: 1.88254e+09
[8]	valid_0's rmse: 40952.7	valid_0's l2: 1.67713e+09
[9]	valid_0's rmse: 40925.8	valid_0's l2: 1.67492e+09
[10]	valid_0's rmse: 38804	valid_0's l2: 1.50575e+09
[11]	valid_0's rmse: 37125.8	valid_0's l2: 1.37833e+09
[12]	valid_0's rmse: 35666.8	valid_0's l2: 1.27212e+09
[13]	valid_0's rmse: 35649	valid_0's l2: 1.27085e+09
[14]	valid_0's rmse: 34475.6	valid_0's l2: 1.18856e+09
[15]	valid_0's rmse: 33583.6	valid_0's l2: 1.12786e+09
[16]	valid_0's rmse: 33571.9	valid_0's l2: 1.12707e+09
[17]	valid_0's rmse: 33563	valid_0's l2: 1.12647e+09
[18]	valid_0's rms

[I 2022-05-15 04:24:23,397] Trial 98 finished with value: 28148.81722627498 and parameters: {'n_estimators': 1673, 'learning_rate': 0.2, 'max_depth': 1, 'subsample': 0.8, 'colsample_bytree': 0.5}. Best is trial 38 with value: 28035.228426059108.
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:563: UserWarning: The distribution is specified by [100, 5000] and step=13, but the range is not divisible by `step`. It will be replaced by [100, 4988].
  low=low, old_high=old_high, high=high, step=step


[1]	valid_0's rmse: 61437.6	valid_0's l2: 3.77458e+09
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's rmse: 61388.7	valid_0's l2: 3.76857e+09
[3]	valid_0's rmse: 47718.2	valid_0's l2: 2.27703e+09
[4]	valid_0's rmse: 47683.7	valid_0's l2: 2.27374e+09
[5]	valid_0's rmse: 39150.5	valid_0's l2: 1.53276e+09
[6]	valid_0's rmse: 34042.4	valid_0's l2: 1.15888e+09
[7]	valid_0's rmse: 31193.6	valid_0's l2: 9.73041e+08
[8]	valid_0's rmse: 29664.5	valid_0's l2: 8.79985e+08
[9]	valid_0's rmse: 29664.3	valid_0's l2: 8.79969e+08
[10]	valid_0's rmse: 28886.2	valid_0's l2: 8.34413e+08
[11]	valid_0's rmse: 28487.6	valid_0's l2: 8.11544e+08
[12]	valid_0's rmse: 28327.4	valid_0's l2: 8.02441e+08
[13]	valid_0's rmse: 28227.8	valid_0's l2: 7.96808e+08
[14]	valid_0's rmse: 28179.9	valid_0's l2: 7.94108e+08
[15]	valid_0's rmse: 28162.7	valid_0's l2: 7.93139e+08
[16]	valid_0's rmse: 28157.1	valid_0's l2: 7.9282e+08
[17]	valid_0's rmse: 28157.1	valid_0's l2: 7.9282e+08
[18]	valid_0's

[I 2022-05-15 04:24:24,089] Trial 99 finished with value: 28122.82909706993 and parameters: {'n_estimators': 2570, 'learning_rate': 0.30000000000000004, 'max_depth': 3, 'subsample': 0.8, 'colsample_bytree': 0.6000000000000001}. Best is trial 38 with value: 28035.228426059108.



[31]	valid_0's rmse: 28164.7	valid_0's l2: 7.93251e+08
[32]	valid_0's rmse: 28158.1	valid_0's l2: 7.92876e+08
[33]	valid_0's rmse: 28157.3	valid_0's l2: 7.92836e+08
[34]	valid_0's rmse: 28158.6	valid_0's l2: 7.92906e+08
[35]	valid_0's rmse: 28161.3	valid_0's l2: 7.93057e+08
[36]	valid_0's rmse: 28163.1	valid_0's l2: 7.93159e+08
[37]	valid_0's rmse: 28164.4	valid_0's l2: 7.93236e+08
[38]	valid_0's rmse: 28166.9	valid_0's l2: 7.93373e+08
[39]	valid_0's rmse: 28164.3	valid_0's l2: 7.93229e+08
[40]	valid_0's rmse: 28164.8	valid_0's l2: 7.93257e+08
[41]	valid_0's rmse: 28166.4	valid_0's l2: 7.93346e+08
[42]	valid_0's rmse: 28160.5	valid_0's l2: 7.93015e+08
[43]	valid_0's rmse: 28188.4	valid_0's l2: 7.94588e+08
[44]	valid_0's rmse: 28188.5	valid_0's l2: 7.94592e+08
[45]	valid_0's rmse: 28199.1	valid_0's l2: 7.95192e+08
[46]	valid_0's rmse: 28199.5	valid_0's l2: 7.9521e+08
[47]	valid_0's rmse: 28195.2	valid_0's l2: 7.94971e+08
[48]	valid_0's rmse: 28187.9	valid_0's l2: 7.94559e+08
[49]	valid

In [25]:
# Fitting the best model

model = lgb.LGBMRegressor(**study.best_params)
model.fit(X_train, y_train)

LGBMRegressor(colsample_bytree=0.6000000000000001, learning_rate=0.2,
              max_depth=2, n_estimators=1777, subsample=0.8)

In [26]:
# Scoring the partitions

print(f"RMSE of Train is {mean_squared_error(model.predict(X_train), y_train, squared=False)}")
print(f"RMSE of Valid is {mean_squared_error(model.predict(X_valid), y_valid, squared=False)}")

RMSE of Train is 26049.211723158838
RMSE of Valid is 28317.554096808333


In [27]:
# Generate the submission file

test = pd.DataFrame({'price': model.predict(X_test)})
test.to_csv('submission.csv', index=False)